# Imports


In [11]:
library(tidyverse)
library(data.table)

# Read data

In [12]:
df_raw = readxl::read_excel('./data/7.3 Camila 2  dados uréia urinária.xlsx')

head(df_raw)

New names:
* `` -> ...324


número,D_IRA,idade,cor,peso,altura,imc,has,dm,icc,...,uU/uCr (g/L/g/L) D7,uNa/uCr (mEq/l/g/L) D7,UV Na 6h (mEq) D7,UV Na 24h (mEq) D7,uNa+uK (mEq/L) D7,uU (mmol/L) D7,(uNa+uK) X 2 + uU (mmol/L) D7,FENa (%) D7,FEK (%) D7,sU/sCr (mg/dL/mg/dL) D7
4,D2,51,1,58.5,1.75,19.10204,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,D2,82,1,72.0,1.75,23.51020,1,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
23,D2,50,1,110.0,1.70,38.06228,1,1,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
26,D2,79,1,46.0,1.55,19.14672,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
31,D2,26,1,60.0,1.75,19.59184,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
46,D2,54,2,55.0,1.65,20.20202,1,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [13]:
categorical_columns = c('D_IRA', 'cor', 'has', 'dm', 'icc',
                        'asma_dpoc', 'hiv', 'ca_ativo', 'avc', 
                        'dvp', 'drogadicao', 'ventilacao_mecanica',
                        'diureticos', 'vasopressina', 'iSRAA',
                        'Causa da IRA / Exposição', 'KDIGO'
                        )

missing_categorical_columns = c('criterio_IRA', 'pos_operatorio', 'mortalidade_intra_hospitalar')

daily_columns = c('Diurese ml/kg/h',
                  'BH (mL/24h)',
                  'SOFA',
                  'sU (mg/dL)',
                  'sCr (mg/dL)',
                  'sNa (mEq/L)',
                  'sK (mEq/L)',
                  'sOsm (mOsm/L)',
                  'uU (g/L)',
                  'uCr (g/L)',
                  'uNa (mEq/L)',
                  'uK (mEq/L)',
                  'Volume urinário (mL/24h)',
                  'UV U6h (g)',
                  'UV U24h (g)',
                  'FEU (%)',
                  'uU/uCr (g/L/g/L)',
                  'uNa/uCr (mEq/L/g/L)',
                  'UV Na 6h (mEq)',
                  'UV Na 24h (mEq)',
                  'uOsm (mOsm/Kg)',
                  'uNa+uK (mEq/L)',
                  'uU (mmol/L)',
                  '(uNa+uK) X 2 + uU (mmol/L)',
                  'FENa (%)',
                  'FEK (%)',
                  'sU/sCr (mg/dL/mg/dL)')
                  

missing_daily_columns = c('uU/sU',
                          'sCrIRA_delta_sCrbasal')

# Wrangling

In [14]:
columns = colnames(df_raw)
una_ucr_cols = columns[grepl(paste0("^", gsub("([\\(\\)])", "\\\\\\1", 'uNa/uCr')), columns)]
setnames(df_raw, old = una_ucr_cols, new = str_replace(una_ucr_cols, 'l', 'L'))

df = df_raw %>%
    mutate_at(categorical_columns, funs(factor(.))) %>%
    mutate(IRA = factor(ifelse(D_IRA == 'SEM IRA', 0, 1)),
           KDIGO = factor(KDIGO),
           KDIGO_agrupado = case_when(KDIGO %in% c(2, 3) ~ "MODERADO/GRAVE",
                                      KDIGO == 1 ~ "LEVE",
                                      KDIGO == 0 ~ "SEM IRA")) %>%
    rename(`BH (mL/24h) D-2` = `BH  (ml/24h) D-2`,
           `uU/uCr (g/L/g/L) D-1` = `uU/uCr (g/L /g/L) D-1`,
           `BH (mL/24h) D-4` = `BH (mL/24h)D-4`,
           `uNa/uCr (mEq/L/g/L) D-1` = `uNa/uCr (mEq/L/g/L)D-1`,
           `uU/uCr (g/L/g/L) D-3` = `uU/uCr (g/L/g/L)D-3`)

df %>%
    head()

número,D_IRA,idade,cor,peso,altura,imc,has,dm,icc,...,UV Na 6h (mEq) D7,UV Na 24h (mEq) D7,uNa+uK (mEq/L) D7,uU (mmol/L) D7,(uNa+uK) X 2 + uU (mmol/L) D7,FENa (%) D7,FEK (%) D7,sU/sCr (mg/dL/mg/dL) D7,IRA,KDIGO_agrupado
4,D2,51,1,58.5,1.75,19.10204,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE
7,D2,82,1,72.0,1.75,23.51020,1,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE
23,D2,50,1,110.0,1.70,38.06228,1,1,1,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE
26,D2,79,1,46.0,1.55,19.14672,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,MODERADO/GRAVE
31,D2,26,1,60.0,1.75,19.59184,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE
46,D2,54,2,55.0,1.65,20.20202,1,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE


In [15]:
for (col in daily_columns){
    df = df %>%
        mutate(!!col := if_else(IRA == 1, !!sym(paste(col, 'D-1')), 
                                !!sym(paste(col, 'Média D1-D5'))))
}

df %>% select(daily_columns)

Diurese ml/kg/h,BH (mL/24h),SOFA,sU (mg/dL),sCr (mg/dL),sNa (mEq/L),sK (mEq/L),sOsm (mOsm/L),uU (g/L),uCr (g/L),...,uNa/uCr (mEq/L/g/L),UV Na 6h (mEq),UV Na 24h (mEq),uOsm (mOsm/Kg),uNa+uK (mEq/L),uU (mmol/L),(uNa+uK) X 2 + uU (mmol/L),FENa (%),FEK (%),sU/sCr (mg/dL/mg/dL)
0.8000000,932.0000,3.0000000,59.00000,1.7200000,144.0000,4.600000,297.8333,7.800000,0.8300000,...,128.915663,29.96000,119.84000,483.0000,133.0000,129.95147,395.95147,1.53982597,11.712939,34.30233
1.4400000,-670.0000,3.0000000,54.00000,0.6300000,127.0000,3.200000,263.0000,11.700000,1.5900000,...,14.465409,14.26000,57.04000,598.0000,111.0000,194.92720,416.92720,0.07175754,10.896226,85.71429
0.7196970,739.0000,3.0000000,133.00000,0.9300000,139.0000,4.500000,300.1667,9.940000,0.5420000,...,33.579336,8.64500,34.58000,349.0000,78.7000,165.60482,323.00482,0.22466750,23.068881,143.01075
1.2681159,228.0000,5.0000000,55.00000,0.6900000,135.0000,3.500000,279.1667,8.090000,0.7950000,...,121.761006,33.88000,135.52000,367.0000,136.1000,134.78299,406.98299,0.62233403,9.745553,79.71014
0.5555556,-82.0000,2.0000000,27.00000,0.9100000,141.0000,4.600000,286.5000,16.420000,2.1010000,...,10.423608,4.38000,17.52000,669.0000,137.2000,273.56450,547.96450,0.06727293,10.856424,29.67033
1.8181818,-314.0000,3.0000000,24.00000,0.6100000,147.0000,4.000000,298.0000,26.580000,1.0734000,...,58.226197,37.50000,150.00000,899.0000,123.4000,442.83461,689.63461,0.24161891,8.652180,39.34426
0.9493671,-891.0000,4.0000000,38.00000,0.7100000,129.0000,3.300000,264.3333,17.240000,0.8427000,...,87.575650,33.21000,132.84000,592.0000,109.0000,287.22606,505.22606,0.48200551,8.986986,53.52113
0.6944444,950.0000,3.0000000,18.00000,0.8400000,139.0000,4.600000,281.0000,3.342000,1.4250000,...,72.280702,25.75000,103.00000,323.0000,171.0000,55.67920,397.67920,0.43680424,8.713959,21.42857
0.5169562,476.0000,2.0000000,37.00000,0.9900000,141.0000,4.200000,288.1667,9.566000,0.6020000,...,20.598007,3.10000,12.40000,211.0000,28.7000,159.37381,216.77381,0.14462430,6.382297,37.37374
1.2500000,230.0000,4.0000000,21.00000,0.5800000,143.0000,5.000000,289.5000,15.193000,1.2180000,...,39.901478,29.16000,116.64000,549.0000,99.6000,253.12213,452.32213,0.16183816,4.857143,36.20690


In [16]:
df = df %>%
    select(-matches("D-?\\d"))

dim(df)

[1] 55 56

In [19]:
head(df)

número,D_IRA,idade,cor,peso,altura,imc,has,dm,icc,...,uNa/uCr (mEq/L/g/L),UV Na 6h (mEq),UV Na 24h (mEq),uOsm (mOsm/Kg),uNa+uK (mEq/L),uU (mmol/L),(uNa+uK) X 2 + uU (mmol/L),FENa (%),FEK (%),sU/sCr (mg/dL/mg/dL)
4,D2,51,1,58.5,1.75,19.10204,0,0,0,...,128.91566,29.960,119.84,483,133.0,129.9515,395.9515,1.53982597,11.712939,34.30233
7,D2,82,1,72.0,1.75,23.51020,1,0,0,...,14.46541,14.260,57.04,598,111.0,194.9272,416.9272,0.07175754,10.896226,85.71429
23,D2,50,1,110.0,1.70,38.06228,1,1,1,...,33.57934,8.645,34.58,349,78.7,165.6048,323.0048,0.22466750,23.068881,143.01075
26,D2,79,1,46.0,1.55,19.14672,0,0,0,...,121.76101,33.880,135.52,367,136.1,134.7830,406.9830,0.62233403,9.745553,79.71014
31,D2,26,1,60.0,1.75,19.59184,0,0,0,...,10.42361,4.380,17.52,669,137.2,273.5645,547.9645,0.06727293,10.856424,29.67033
46,D2,54,2,55.0,1.65,20.20202,1,0,0,...,58.22620,37.500,150.00,899,123.4,442.8346,689.6346,0.24161891,8.652180,39.34426


In [20]:
names(df)

[1] "número"                     "D_IRA"                     
 [3] "idade"                      "cor"                       
 [5] "peso"                       "altura"                    
 [7] "imc"                        "has"                       
 [9] "dm"                         "icc"                       
[11] "asma_dpoc"                  "hiv"                       
[13] "ca_ativo"                   "avc"                       
[15] "dvp"                        "drogadicao"                
[17] "score_clinico"              "ventilacao_mecanica"       
[19] "dva_mcg_kg_min"             "diureticos"                
[21] "vasopressina"               "iSRAA"                     
[23] "saps3"                      "egfr_basal"                
[25] "Causa da IRA / Exposição"   "KDIGO"                     
[27] "...324"                     "IRA"                       
[29] "KDIGO_agrupado"             "Diurese ml/kg/h"           
[31] "BH (mL/24h)"                "SOFA"                      
[33] "sU (mg/dL)"                 "sCr (mg/dL)"               
[35] "sNa (mEq/L)"                "sK (mEq/L)"                
[37] "sOsm (mOsm/L)"              "uU (g/L)"                  
[39] "uCr (g/L)"                  "uNa (mEq/L)"               
[41] "uK (mEq/L)"                 "Volume urinário (mL/24h)"  
[43] "UV U6h (g)"                 "UV U24h (g)"               
[45] "FEU (%)"                    "uU/uCr (g/L/g/L)"          
[47] "uNa/uCr (mEq/L/g/L)"        "UV Na 6h (mEq)"            
[49] "UV Na 24h (mEq)"            "uOsm (mOsm/Kg)"            
[51] "uNa+uK (mEq/L)"             "uU (mmol/L)"               
[53] "(uNa+uK) X 2 + uU (mmol/L)" "FENa (%)"                  
[55] "FEK (%)"                    "sU/sCr (mg/dL/mg/dL)"

In [18]:
write.csv(df, "./data/processed_data_mean.csv", row.names=FALSE, quote=FALSE) 

In [21]:
saveRDS(df, "./data/processed_data_mean.rds")